In [30]:
%load_ext autoreload
%autoreload 2

import os
print(os.getcwd())
import sys
sys.path.append(os.path.abspath(".."))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/root/code/Vivi-dsz/value_mapping/value_mapping/notebooks


In [31]:
import pandas as pd
from backend.preprocess.data_prep import *
from backend.preprocess.params import *
from backend.preprocess.alignment_score import get_alignment_summary
from backend.preprocess.brand_count_function import get_brand_counts
from backend.preprocess.review_count_function import get_review_counts
from data.raw.brands_about_us import brand_text
from data.raw.brandcompareinfo import brand_comparison_info
from backend.models.agent import *


In [22]:
file_path = '/root/code/Vivi-dsz/value_mapping/value_mapping/data/raw/user_reviews_v01.csv'
reviews_df_full = pd.read_csv(file_path)
reviews_df_full.head()


,reviewId,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,app
0,f97ba6d2-4d7c-4aab-ae45-a847584972cf,was using stocard everyday. without any notice...,1,0,25.29.239,2025-07-27 12:10:19,Hello and thank you for your review.\nThe loya...,2025-07-27 12:30:43,25.29.239,Klarna
1,e325f04e-0b7b-45ae-9013-a01a016baeab,Zero (0) of my cards transfered from Stocard. ...,1,0,25.29.239,2025-07-27 12:03:24,Thank you for the review. we understand that y...,2025-07-15 23:49:02,25.29.239,Klarna
2,f26beb63-d371-4759-8af7-cf4e0bf7421f,Nobody should use it. They ruined(acquired) th...,1,0,25.29.239,2025-07-27 06:28:26,Thank you for the review. We understand that y...,2025-07-27 06:49:25,25.29.239,Klarna
3,eed09a6f-c72a-4662-bd81-8d19d73a01d6,This Klarna app is garbage. When they switched...,1,0,25.29.239,2025-07-27 05:22:54,Thanks for the review. We are sorry to hear th...,2025-07-27 05:43:29,25.29.239,Klarna
4,1bcfeb1b-ca54-4883-9c12-34df63e6e8c4,"Absolutely horrendous, cannot transfer years o...",1,0,NaN,2025-07-27 01:22:31,Thank you for the review. We try to innovate o...,2025-07-27 01:41:27,NaN,Klarna


In [4]:
file_path = '/root/code/Vivi-dsz/value_mapping/value_mapping/data/preprocessed/topics_newkw_thousand.csv'


reviews_df =pd.read_csv(
    file_path,
    delimiter=';',
    encoding='ISO-8859-1',
    on_bad_lines='skip',  # Optional: skip malformed lines
    engine='python'  # Required when using `on_bad_lines` with older pandas
)

reviews_df

,Column1,reviewId,content,topic_openai,topic_category
0,0,00885280-2fda-46bf-9c24-142c644ab106,Used to be great. They changed my card and I h...,trust,trust_ethics
1,1,962be690-af75-4f76-94d4-76328aeeab83,"Very expensive, the plan costs more than brick...",affordable,quality_usability
2,2,a1730aab-1933-4b0c-8476-038583b54903,First I give 1 star because I had problem with...,service,user_centricity_support
3,3,7c32fad5-4eed-496d-bc51-a34905b8a22c,Great application It really work for me,functionality,quality_usability
4,4,aa8542cc-3282-4dd0-911c-dddac3d3a844,"Very good service, thank you.",service,user_centricity_support
...,...,...,...,...,...
995,995,0ec7e0a5-a70b-41e2-bcfd-95bd8c4142a7,Amazing app,experience,user_centricity_support
996,996,75d58234-4f4b-4650-a60b-9045211af444,gets stuck too often,reliability,quality_usability
997,997,64732b17-1a79-4244-8f42-a60d8575d16f,Al tratar de ingresar dinero la app crashea. A...,reliability,quality_usability
998,998,74941c3f-58c6-466c-9e3b-7d7ddbc369e0,Looking good,quality,quality_usability


In [5]:
# Merge the review_df with the reviews_df_full on reviewId to get apps

review_merged_df = pd.merge(
    reviews_df,
    reviews_df_full[['reviewId', 'app']],
    on='reviewId',
    how='left'
)


review_merged_df['app'] = review_merged_df['app'].replace({"TradeRepublic": "Trade Republic"})

In [6]:
review_merged_df.head()

,Column1,reviewId,content,topic_openai,topic_category,app
0,0,00885280-2fda-46bf-9c24-142c644ab106,Used to be great. They changed my card and I h...,trust,trust_ethics,Bunq
1,1,962be690-af75-4f76-94d4-76328aeeab83,"Very expensive, the plan costs more than brick...",affordable,quality_usability,Bunq
2,2,a1730aab-1933-4b0c-8476-038583b54903,First I give 1 star because I had problem with...,service,user_centricity_support,Bunq
3,3,7c32fad5-4eed-496d-bc51-a34905b8a22c,Great application It really work for me,functionality,quality_usability,Bunq
4,4,aa8542cc-3282-4dd0-911c-dddac3d3a844,"Very good service, thank you.",service,user_centricity_support,Bunq


In [7]:
keywords = [
    word
    for category, words in values_keywords.items()
    for word in [category] + words
]

len(keywords)

95

In [8]:
keywords = [
    word
    for words in values_keywords.values()
    for word in words
]

len(keywords)

87

In [9]:
brand_kw_df = get_brand_counts(brand_text, keywords)
review_kw_df = get_review_counts(review_merged_df, keywords)


In [10]:
print(brand_kw_df.shape)
print(review_kw_df.shape)


(5, 39)
(5, 33)


In [11]:
brand_kw_df.head()

,secure,growth,impact,smart,digital,accountability,transparency,responsibility,safety,control,...,social,ambition,empowerment,freedom,automation,ease,innovation,intuitive,reliability,flexibility
brand,,,,,,,,,,,,,,,,,,,,,
Klarna,5.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Revolut,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bunq,0.0,0.0,5.0,2.0,2.0,1.0,1.0,0.0,0.0,6.0,...,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
N26,10.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,3.0,...,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0
Trade Republic,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


In [12]:
list(review_merged_df['app'].unique())

['Bunq', 'Klarna', 'N26', 'Revolut', 'Trade Republic']

In [13]:
review_kw_df.head()

,support,community,control,ai,trust,speed,reliability,digital,safety,smooth,...,innovation,functional,simplicity,ethics,smart,affordable,flexibility,integrity,impact,planning
brand,,,,,,,,,,,,,,,,,,,,,
Bunq,42.0,1.0,2.0,14.0,3.0,1.0,1.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Klarna,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N26,67.0,0.0,0.0,2.0,3.0,2.0,1.0,3.0,0.0,2.0,...,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0
Revolut,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0
Trade Republic,54.0,0.0,0.0,4.0,3.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [14]:
brand_kw_df.head()

,secure,growth,impact,smart,digital,accountability,transparency,responsibility,safety,control,...,social,ambition,empowerment,freedom,automation,ease,innovation,intuitive,reliability,flexibility
brand,,,,,,,,,,,,,,,,,,,,,
Klarna,5.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Revolut,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Bunq,0.0,0.0,5.0,2.0,2.0,1.0,1.0,0.0,0.0,6.0,...,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
N26,10.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,0.0,3.0,...,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0
Trade Republic,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


In [15]:
for brand in list(review_merged_df['app'].unique()):
    summary = get_alignment_summary(brand, brand_kw_df, review_kw_df)
    print(f"{brand}: {summary}")

Bunq: {'brand': 'Bunq', 'Totall alignment_score': 0.452, 'Top values alignment score: ': 0.333, 'top_brand_values': ['user_centricity', 'empowerment', 'growth'], 'top_user_values': ['user_centricity', 'ethos', 'innovation'], 'misaligned_values': ['empowerment', 'growth', 'ethos', 'innovation'], 'recommendation': "Bunq emphasizes empowerment, growth in its messaging, but these values are not strongly perceived by users.Consider reinforcing them in UX, marketing, or customer service.Users perceive ethos, innovation as important, but these are not highlighted in the brand's communication.Consider reinforcing them in the brand communication."}
Klarna: {'brand': 'Klarna', 'Totall alignment_score': 0.156, 'Top values alignment score: ': 0.667, 'top_brand_values': ['user_centricity', 'quality'], 'top_user_values': ['user_centricity', 'quality', 'simplicity'], 'misaligned_values': ['simplicity'], 'recommendation': 'Klarna is well-aligned in its brand values and user perception.'}
N26: {'brand'

## Use GPT and Calling Tool Function

In [32]:
question = 'How can Klarna win over N26?'
response = handle_query(question, brand_kw_df, review_kw_df, api_key = os.getenv("OpenAI_API_KEY"))
print(response)

# question = 'What kind of campaign should Klarna run next?'
# response = handle_query(question, brand_kw_df, review_kw_df)
# print(response)

['Klarna', 'N26']
To effectively position Klarna against N26 in the German market, let's analyze the data and derive strategic insights:

### Key Differences and Positioning:

1. **Value Alignment:**
   - **Klarna** has a strong alignment with users on "user_centricity" and "quality," but there's a noticeable gap in "simplicity."
   - **N26** is misaligned on "innovation," "ethos," and "simplicity," despite these being perceived as important by users. This indicates a gap in N26's communication and user expectations.

2. **Sentiment Trends:**
   - **Klarna** shows fluctuating sentiment in "empowerment_control" and "quality_usability," with recent declines in "trust_ethics" and "user_centricity_support." This suggests potential risks in user trust and support.
   - **N26** also faces challenges in "trust_ethics" and "user_centricity_support," with consistently negative sentiment, indicating a need for improvement in these areas.

3. **Strategic Risks and Opportunities:**
   - **Klarna**